# Prompt engineering

In [6]:
import os
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

print("API Key:", api_key)

client = Mistral(api_key=api_key)

chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)
print(chat_response.choices[0].message.content) # type: ignore

API Key: BCvosXYEve3u1dqqI4cmDTtxjDiQuC8K
Determining the "best" French cheese can be quite subjective, as it depends on personal taste. France is renowned for its wide variety of cheeses, with estimates suggesting there are over 1,600 different types. Here are a few highly regarded French cheeses across various categories:

1. **Soft Cheeses**:
   - **Brie de Meaux**: Often referred to as the "King of Cheeses," it has a rich, creamy interior and a bloomy rind.
   - **Camembert de Normandie**: Another famous soft cheese with a creamy center and a white, edible rind.

2. **Semi-Soft Cheeses**:
   - **Reblochon**: Known for its nutty taste and smooth, supple interior.
   - **Morbier**: Recognizable by its layer of ash in the middle, with a mild, slightly nutty flavor.

3. **Hard Cheeses**:
   - **Comté**: A firm, pressed cheese with a complex flavor profile that includes notes of fruit, nuts, and spices.
   - **Beaufort**: Similar to Comté, it has a firm texture and a rich, nutty flavor.

# Mistral Chat Completion API

In [10]:
def get_completion(prompt, model="mistral-small-latest"):
    messagesa = [
        {"role": "user",
          "content": prompt
          }
          ]
    response = client.chat.complete(
        model=model,
        messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

# Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input

In [11]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To get the desired output from a model, provide clear, specific, and often detailed instructions to guide it and minimize irrelevant or incorrect responses.


#### Tactics 2 - Ask for a structured output¶

In [14]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

```json
[
  {
    "book_id": 1,
    "title": "Whispers of the Starlit Abyss",
    "author": "Eleanor Nightingale",
    "genre": "Science Fiction"
  },
  {
    "book_id": 2,
    "title": "The Enchanted Forest Chronicles",
    "author": "Oliver Greenwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 3,
    "title": "Echoes of the Forgotten Melody",
    "author": "Isabella Hartley",
    "genre": "Mystery"
  }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [15]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.


In [16]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [17]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The mighty oak that withstands the storm \
began as a tiny acorn; the phoenix that rises from the ashes \
first endured the fire; the mountain that touches the sky \
was once a mere pebble, shaped by time and pressure.

<child>: Teach me about courage.

<grandparent>: The smallest bird that dares to sing in the \
darkest night brings hope to all who listen; the lone \
soldier who stands against the tide can inspire a \
revolution; the quiet voice that speaks the truth in the \
face of adversity can change the world.


# Principle 2: Give the model time to “think” 

## Tactic 1: Specify the steps required to complete a task

In [18]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Siblings Jack and Jill embark on a quest to fetch water from a hilltop well, encounter a mishap, and return home slightly battered but undeterred.

Les frères et sœurs Jack et Jill entreprennent une quête pour aller chercher de l'eau à un puits en haut d'une colline, rencontrent un accident et rentrent chez eux légèrement meurtris mais indomptés.

Jack, Jill

```json
{
  "french_summary": "Les frères et sœurs Jack et Jill entreprennent une quête pour aller chercher de l'eau à un puits en haut d'une colline, rencontrent un accident et rentrent chez eux légèrement meurtris mais indomptés.",
  "num_names": 2
}
```


## specific output

In [19]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Siblings Jack and Jill embark on a quest to fetch water but fall down a hill, returning home slightly battered but still adventurous.
Translation: Les frères et sœurs Jack et Jill entreprennent une quête pour chercher de l'eau mais tombent d'une colline, rentrant chez eux légèrement meurtris mais toujours aventureux.
Names: Jack, Jill
Output JSON: {"french_summary": "Les frères et sœurs Jack et Jill entreprennent une quête pour chercher de l'eau mais tombent d'une colline, rentrant chez eux légèrement meurtris mais toujours aventureux.", "num_names": 2}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [20]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To determine the total cost for the first year of operations as a function of the number of square feet, we need to consider the following costs:

1. **Land cost**: $100 per square foot
2. **Solar panel cost**: $250 per square foot
3. **Maintenance cost**: $100,000 flat fee per year + $10 per square foot

Let \( x \) be the size of the installation in square feet.

1. **Land cost**: \( 100x \)
2. **Solar panel cost**: \( 250x \)
3. **Maintenance cost**: \( 100,000 + 10x \)

Now, we add all these costs together to get the total cost for the first year:

\[ \text{Total cost} = 100x + 250x + 100,000 + 10x \]

Combine like terms:

\[ \text{Total cost} = (100x + 250x + 10x) + 100,000 \]
\[ \text{Total cost} = 360x + 100,000 \]

So, the total cost for the first year of operations as a function of the number of square feet is:

\[ \text{Total cost} = 360x + 100,000 \]

Is the student's solution the same as the actual solution just calculated:
```
no
```

Student grade:
```
incorrect
```
